# Sparse state networks 
Second-order dynamics on a physical network can be described by first-order dynamics on a second-order state network.

We can represent this second-order network by it's _state transition matrix_ $P_{ij}$ with the probabilities for the random walker to transition from state node $i$ to state node $j$.

In this view, we may note that some rows have similar probability distributions. We can measure how much information we lose when merging two state nodes with the [Jensen-Shannon Distance](https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence).

The idea behind sparse state networks is that we can lump state nodes (within each physical node) that constrain the network flow in a similar way without losing (much) information.

## Transform to a general machine learning problem
We will here solve the problem using a divisive clustering algorithm in [clustering_algorithm.py](./clustering_algorithm.py).

In order to do that, we have to transform the state network into features usable for machine learning. We can do this with the help of the code in [state_network.py](./state_network.py).

**TODO**
- Import StateNetwork from state_network
- Create a new StateNetwork using the `.from_paths(filename)` method to read in `../data/toy_paths.net`

In [1]:
from state_network import StateNetwork

order = 2
net, _ = StateNetwork.from_paths("../data/toy_paths.net", output_filename="../output/toy_states.net", markov_order=order)
net.aggregate(order)
print(net)

Read path data from file '../data/toy_paths.net'...
Done, parsed 32/32 paths
 -> 12 return paths
 -> 12 states in training network
Writing state network to file '../output/toy_states.net'...
<StateNetwork phys_nodes=[
	<PhysNode id=5 containers=<Cluster id=0 state_ids=[1,9]>>
	<PhysNode id=4 containers=<Cluster id=1 state_ids=[2,11]>>
	<PhysNode id=1 containers=<Cluster id=2 state_ids=[3,7,10,12]>>
	<PhysNode id=2 containers=<Cluster id=3 state_ids=[4,8]>>
	<PhysNode id=3 containers=<Cluster id=4 state_ids=[5,6]>>
/>


<img src="../figures/toy_states.png" width="450">

Figure 1: Second-order state network in `output/toy_states.net`

## Feature matrix

The feature matrix for a physical node is simply the rows of the state transition matrix for the state nodes belonging to that physical node.

To simplify, there is a `get_feature_matrix` method that removes all all-zero rows and columns in the feature matrix and provides a mapping back to the original state network. It returns a tuple `(X, T)`, where `X` is the feature matrix (np.array) of size (numNonDanglingStateNodes, numLinkedNodes) and `T` is a dictionary transforming row index in the feature matrix to state node id.

**TODO**
- Use the method above and get the feature matrix and rowToStateId map
- Print the two items

In [2]:
central_node = net.phys_nodes[1]
X, row_index_to_state_ids = net.get_feature_matrix(central_node, order)

print(f"Feature matrix for the central physical node:\n{X}")
print(f"row_index_to_state_ids:\n{row_index_to_state_ids}")

Feature matrix for the central physical node:
[[0.39690536 0.39174763 0.10663308 0.10471393]
 [0.10174558 0.10150646 0.40124342 0.39550454]
 [0.39658623 0.39976457 0.09535021 0.108299  ]
 [0.10601476 0.09681714 0.3971923  0.3999758 ]]
row_index_to_state_ids:
{0: [3], 1: [7], 2: [10], 3: [12]}


### Measure pairwise similarity
Now we can compare rows pairwise and cluster the most similar rows together. The Jensen-Shannon distance is unfortunately not implemented in scikit-learn (though it exist in a [pull request](https://github.com/scikit-learn/scikit-learn/pull/4191)), so let's create it.

**TODO**
- Write a function that takes two equally sized arrays of probabilities as input and returns the Jensen-Shannon distance between them
- Compute the Jensen-Shannon distance between the two different rows of the feature matrix, and check that at gives zero for same input

Tips, using numpy:
- Work with `np.asarray(x)` in the function to allow for both a numpy array and an ordinary python list as input
- `np.log2(x)` can be modified to `np.log2(x, where = x>0)` to handle zeros

In [3]:
import numpy as np

def plogp(x):
    x = np.asarray(x)
    return x * np.log2(x, where = x>0)

def entropy(x):
    return -np.sum(plogp(x))

def jensen_shannon_distance(x1, x2):
    x1 = np.asarray(x1)
    x2 = np.asarray(x2)
    mix = (x1 + x2) / 2
    return np.sqrt(entropy(mix) - (entropy(x1) + entropy(x2)) / 2)

print(jensen_shannon_distance(X[0], X[0]))
print(jensen_shannon_distance(X[0], X[1]))

0.0
0.5135537698931543


### Cluster with divisive clustering algorithm

Now we can use the divisive clustering algorithm that uses the pairwise distance function as a metric. It can take as input the number of clusters you want. For the example feature matrix, it's two.

**TODO**
- Create a JSdivisiveClustering model and find two clusters based on the Jensen-Shannon distance
- Use the row-to-stateId map to check which state nodes are clustered together (the left ones are state node 7 and 12, the right ones are state node 3 and 10).

In [4]:
from clustering_algorithm import JSdivisiveClustering

labels = JSdivisiveClustering(X, net.tot_weight).divide_labels(n_clusters=2)

label_to_state_ids = {label: [] for label in labels}

for row_index, label in enumerate(labels):
    label_to_state_ids[label].extend(row_index_to_state_ids[row_index])

print(label_to_state_ids)

{0: [3, 10], 1: [7, 12]}


### Lump whole network
Now we are ready to run this on the whole network. For convenience, `StateNetwork` provides a method `cluster_state_nodes` that takes an argument `js_div_threshold`. This will divide clusters as long as the information gain is greater than the threshold.

**TODO**
- Cluster the whole state network using the method above

In [5]:
net.cluster_state_nodes(order, js_div_threshold=1e-5)
print(net)

<StateNetwork phys_nodes=[
	<PhysNode id=5 containers=<Cluster id=5 state_ids=[1,9]>>
	<PhysNode id=4 containers=<Cluster id=6 state_ids=[2,11]>>
	<PhysNode id=1 containers=<Cluster id=7 state_ids=[3,10]><Cluster id=8 state_ids=[7,12]>>
	<PhysNode id=2 containers=<Cluster id=9 state_ids=[4,8]>>
	<PhysNode id=3 containers=<Cluster id=10 state_ids=[5,6]>>
/>


<img src="../figures/toy_states_lumped.png" width="450">

Figure 2: Sparse state network

## Did we lose any information?
The state network has two properties `entropy_rate` and `lumped_entropy_rate` to calculate the average number of bits required to encode the random walk on each physical node.

**TODO**
- Run the methods above and check that the entropy rates stayed the same

In [6]:
print(f"Entropy rate before: {net.entropy_rate}")
print(f"Entropy rate after:  {net.lumped_entropy_rate}")

Entropy rate before: 1.244833806460494
Entropy rate after:  1.2449413025980005


In [7]:
from pathlib import Path
net.write_clustered_network("../output/toy_states_lumped.net")
print(Path('../output/toy_states_lumped.net').read_text())

Writing clustered state network to file '../output/toy_states_lumped.net'...
# physical nodes: 5
# clustered state nodes: 6
*Vertices
1 "i"
2 "j"
3 "k"
4 "l"
5 "m"
*States
#state_id physical_id 
5 5
6 4
7 1
8 1
9 2
10 3
*Links
#source_id target_id weight
5 6 8374
5 7 8269
6 5 8415
6 7 8293
7 6 6678
7 5 6662
7 10 1699
7 9 1793
8 10 6638
8 9 6613
8 6 1727
8 5 1649
9 10 8443
9 8 8167
10 8 8357
10 9 8223



### Now we have generated the sparse network (with lossless compression)
<img src="../figures/toy_states.png" width="450">
<img src="../figures/toy_states_lumped.png" width="450">

Figure 3: The original second-order network and the sparse network formed by lumping similar state nodes within each physical node.